In [1]:
# data manipulation library
import pandas as pd
import numpy as np
import json

import matplotlib.pyplot as plt
import seaborn as sns

# web scraping libraries
from bs4 import BeautifulSoup
import requests


import plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from collections import Counter
import numpy as np
from sklearn.ensemble import RandomForestRegressor

from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler


pd.set_option('display.max_rows',200)
pd.set_option('display.max_columns',200)

## Data Collection

In [2]:
#loading the at home store details json file
with open('store_details.json') as f:
    store_details = json.load(f)

In [3]:
# getting the store names
store_name = []
for i in store_details['stores']:
    store_name.append(i['ID'])

In [4]:
# getting the store address
store_address = []
for i in store_details['stores']:
    store_address.append(i['address1'])

In [5]:
# getting the city of the stores
store_city = []
for i in store_details['stores']:
    store_city.append(i['city'])

In [6]:
# getting zipcode
store_zipcode = []
for i in store_details['stores']:
    store_zipcode.append(i['postalCode'])

In [7]:
# getting state details
store_state = []
for i in store_details['stores']:
    store_state.append(i['stateCode'])

In [8]:
At_Home_stores = pd.DataFrame(list(zip(store_name, store_address, store_state, store_city, store_zipcode)), columns=['Name','Address','State','City','Zipcode'])

In [9]:
At_Home_stores

,Name,Address,State,City,Zipcode
0,Massachusetts-Dedham,300 Providence Hwy,MA,Dedham,02026
1,Massachusetts-Seekonk,1110 Fall River Ave,MA,Seekonk,02771
2,Rhode Island-Warwick North,650 Bald Hill Rd,RI,Warwick,02886
3,Connecticut-Manchester,1181 Tolland Turnpike,CT,Manchester,06042
4,New York-Albany,161 Washington Ave Ext,NY,Albany,12205
...,...,...,...,...,...
222,California-San Diego,12080 Carmel Mountain Rd,CA,San Diego,92128
223,California-Foothill Ranch,26532 Towne Centre Dr Suites A & B,CA,Foothill Ranch,92610
224,California-Clovis,1075 Shaw Ave,CA,Clovis,93612
225,California-Sacramento,8320 Delta Shores Circle S.,CA,Sacramento,95832


In [10]:
# getting store hours
store_hours = []
for i in range(len(store_details['stores'])):
    try:
        store_hours.append(store_details['stores'][i]['storeHours'])
    except:
        store_hours.append('Not found')
        print(i)
        print(store_details['stores'][i]['ID'])

5
New York-Nanuet
7
New York-Rego Park
9
New Jersey-Ocean Township
20
Pennsylvania-York
26
Virginia-Leesburg
76
Indiana-Clarksville
154
Oklahoma-Oklahoma City North
186
Colorado-Johnstown


In [11]:
curbside = []
for i in range(len(store_hours)):
    if store_hours[i].find('curbside pickup') != -1:
        curbside.append(1)
    else:
        curbside.append(0)

In [12]:
len(curbside)

227

In [13]:
curbside.count(1)

152

In [14]:
curbside.count(0)

75

In [15]:
local_delivery = []
for i in range(len(store_hours)):
    if store_hours[i].find('local delivery') != -1:
        local_delivery.append(1)
    else:
        local_delivery.append(0)

In [16]:
local_delivery.count(1)

133

In [17]:
local_delivery.count(0)

94

In [18]:
At_Home_stores['In_store_pickup'] = 1
At_Home_stores['Curbside_pickup'] = curbside
At_Home_stores['local_delivery_pickup'] = local_delivery

In [19]:
At_Home_stores.sort_values(by = 'State',inplace=True)
At_Home_stores

,Name,Address,State,City,Zipcode,In_store_pickup,Curbside_pickup,local_delivery_pickup
110,Alabama-Huntsville,1401 Memorial Pkwy NW,AL,Huntsville,35816,1,0,0
114,Alabama-Trussville,5036 Pinnacle Square,AL,Trussville,35235,1,1,1
118,Alabama-Hoover,5501 Grove Blvd,AL,Hoover,35226,1,1,1
119,Alabama-Montgomery,1500 Eastdale Mall,AL,Montgomery,36117,1,0,0
136,Alabama-Mobile,312 Schillinger Rd,AL,Mobile,36608,1,0,0
...,...,...,...,...,...,...,...,...
102,Wisconsin-Madison West,7050 Watts Rd,WI,Madison,53719,1,1,1
100,Wisconsin-Madison East,2201 Zeier Rd,WI,Madison,53704,1,1,1
91,Wisconsin-Grand Chute,3820 W Wisconsin Ave,WI,Grand Chute,54914,1,1,1
85,Wisconsin-Wauwatosa,3201 N Mayfair Rd,WI,Wauwatosa,53222,1,1,1


In [20]:
At_Home_stores[At_Home_stores['Zipcode'] == '16565']

,Name,Address,State,City,Zipcode,In_store_pickup,Curbside_pickup,local_delivery_pickup
31,Pennsylvania-Erie,2080 Interchange Rd #500,PA,Erie,16565,1,1,0


In [21]:
# loading the sales data
AHG_sales_data = pd.read_excel('Simulated_Store_Sales_2021Feb23-4.xlsx')

In [22]:
# getting the top 5 character in each zipcode
AHG_sales_data['ZIPCODE'] = AHG_sales_data['ZIPCODE'].apply(str)
AHG_sales_data['ZIPCODE'] = AHG_sales_data['ZIPCODE'].apply(lambda x: x[:5])

In [23]:
AHG_sales_data.head()

,Year,DMA_DESC,STORE ADDRESS,CITY,STATE,ZIPCODE,SIMULATED_SALES
0,2017,Memphis,5280 Summer Ave,Memphis,TN,38122,1.158712e+07
1,2017,Atlanta,1605 Buford Highway,BUFORD,GA,30518,9.483031e+06
2,2017,Toledo,2244 S Reynolds Rd,TOLEDO,OH,43614,7.325231e+06
3,2017,Phoenix (Prescott),5000 S. Arizona Mills Circle,TEMPE,AZ,85282,6.567164e+06
4,2017,Memphis,7697 Winchester Road,Memphis,TN,38125,6.803906e+06


In [24]:
AHG_sales_data.shape

(769, 7)

In [25]:
r = []
for i in AHG_sales_data.ZIPCODE.value_counts().index:
    if len(i) <5:
        r.append(i)
print(f'improper zipcodes {r}')

improper zipcodes ['2886', '8002', '2771', '7470', '8724', '6042', '8012', '2026']


In [26]:
len(r)

8

In [27]:
# removing the improper zipcodes from dataframe
AHG_sales_data = AHG_sales_data[~AHG_sales_data.ZIPCODE.isin(r)]
AHG_sales_data.shape

(743, 7)

In [28]:
AHG_sales_data.Year.value_counts()

2020    211
2019    208
2018    176
2017    148
Name: Year, dtype: int64

In [29]:
zipcodes = AHG_sales_data.ZIPCODE.unique()
len(zipcodes)

218

In [30]:
athome_zip = list(At_Home_stores['Zipcode'].values)
len(athome_zip)

227

In [31]:
all_zip = [*zipcodes , *athome_zip]

In [32]:
all_zip = set(all_zip)
len(all_zip)

238

## Web Scraping using Python

In [33]:
source = requests.get('https://www.uszip.com/zip/35816').text
soup = BeautifulSoup(source, 'lxml')
container1 = soup.find('div', class_ = 'container-fluid')
columns = ['Zipcode']
Q1 = container1.find_all('dt')
for t in Q1:
    columns.append(t.text)

columns.remove('Median age')
columns.remove('Car, truck, or van') 

In [34]:
columns = ['Zipcode', 'Total population', 'Housing units', 'Land area(sq. miles)', 'Density(people per sq. mile)', 'Water area(sq. miles)', 'Male', 'Female', 'Median_Age_Male', 'Median_Age_Female', 'Total households', 'Family households', 'Nonfamily households', 'Average household size', 'Average family size', 'Total housing units', 'Occupied housing units', 'Owner-occupied', 'Renter-occupied', 'Vacant housing units', 'Age Under 5 years', 'Age 5 to 9 years', 'Age 10 to 14 years', 'Age 15 to 19 years', 'Age 20 to 24 years', 'Age 25 to 34 years', 'Age 35 to 44 years', 'Age 45 to 54 years', 'Age 55 to 59 years', 'Age 60 to 64 years', 'Age 65 to 74 years', 'Age 75 to 84 years', 'Age 85 years and over', 'Median age', 'White', 'Hispanic or Latino', 'Black or African American', 'Asian', 'American Indian & Alaska Native', 'Native Hawaiian & Other Pacific Islander', 'Other (One Race)', 'Two or More Races', 'Population(16 years and over)', 'Civilian labor force', 'Employed', 'Unemployed', 'Armed Forces', 'Not in labor force', 'Percent Unemployed', 'Workers(16 years and over)', 'Drove alone', 'Carpooled', 'Public transportation(excluding taxicab)', 'Walked', 'Other means', 'Worked at home', 'Mean travel time', 'Civilian employed population(16 years and over)', 'Management / business / science / arts', 'Service', 'Sales / office', 'Natural resources / construction / maintenance', 'Production / transportation / material moving', 'Civilian employed population(16 years and over)', 'Agriculture / forestry / fishing / hunting / mining', 'Construction', 'Manufacturing', 'Wholesale trade', 'Retail trade', 'Transportation / warehousing / utilities', 'Information', 'Finance / insurance / real estate / rental / leasing', 'Professional / scientific / management / administrative / waste management services', 'Educational services / health care / social assistance', 'Arts / entertainment / recreation / accommodation / food services', 'Public administration', 'Other services', 'Civilian employed population(16 years and over)', 'Private wage & salary workers', 'Government workers', 'Self-employed(in own not incorporated business)', 'Unpaid family workers', 'Total households', 'Income Less than $10,000', 'Income $10,000 to $14,999', 'Income $15,000 to $24,999', 'Income $25,000 to $34,999', 'Income $35,000 to $49,999', 'Income $50,000 to $74,999', 'Income $75,000 to $99,999', 'Income $100,000 to $149,999', 'Income $150,000 to $199,999', 'Income $200,000 or more', 'Median household income', 'Mean household income', 'Percentage of people whose income in the past 12 months is below the poverty level']

In [35]:
len(columns)

96

In [36]:
def scraping(zip_codes):
    
    final_data = {}
    
    for i in range(len(zip_codes)):
        source = requests.get('https://www.uszip.com/zip/' + zip_codes[i]).text
        soup = BeautifulSoup(source, 'lxml')
        container1 = soup.find('div', class_ = 'container-fluid')
                              
        Q2 = container1.find_all('dd')
        value = []
        for t in Q2:
            value.append(t.text.replace('▼','').replace('▲',''))
        if len(value) == 0:
            value = [0] * 95
                              
        final_data[zip_codes[i]] = value

        
    return final_data

In [37]:
#Zip_codes = list(At_Home_stores['Zipcode'].values)
final_data_dict = scraping(list(all_zip))

In [38]:
final_data_dict

{'77479': ['74,514',
  '24,980',
  '33.06',
  '2,253.90',
  '1.55',
  '36,489',
  '38,025',
  '38.60',
  '39.20',
  '24,076',
  '20,738',
  '3,338',
  '3.09',
  '3.38',
  '24,980',
  '24,076',
  '20,367',
  '3,709',
  '904',
  '4,455',
  '6,050',
  '7,022',
  '5,880',
  '3,065',
  '6,758',
  '11,918',
  '14,078',
  '5,630',
  '4,014',
  '3,464',
  '1,546',
  '634',
  '39.00',
  '33,503',
  '6,736',
  '5,100',
  '27,059',
  '124',
  '23',
  '166',
  '1,803',
  '54,403',
  '36,562',
  '34,928',
  '1,634',
  '7',
  '17,834',
  '4.5%',
  '34,397',
  '27,774',
  '3,570',
  '657',
  '115',
  '320',
  '1,961',
  '30.0 min.',
  '34,928',
  '22,157',
  '1,818',
  '8,696',
  '913',
  '1,344',
  '34,928',
  '1,977',
  '1,371',
  '3,058',
  '1,701',
  '3,677',
  '1,571',
  '643',
  '3,500',
  '5,783',
  '8,006',
  '1,611',
  '712',
  '1,318',
  '34,928',
  '28,925',
  '3,758',
  '2,148',
  '97',
  '22,797',
  '507',
  '253',
  '762',
  '947',
  '1,528',
  '2,533',
  '3,117',
  '4,694',
  '3,634',


In [39]:
df = pd.DataFrame(final_data_dict)
final_df = df.T
final_df = final_df.reset_index()

In [40]:
final_df.columns =columns
final_df

,Zipcode,Total population,Housing units,Land area(sq. miles),Density(people per sq. mile),Water area(sq. miles),Male,Female,Median_Age_Male,Median_Age_Female,Total households,Family households,Nonfamily households,Average household size,Average family size,Total housing units,Occupied housing units,Owner-occupied,Renter-occupied,Vacant housing units,Age Under 5 years,Age 5 to 9 years,Age 10 to 14 years,Age 15 to 19 years,Age 20 to 24 years,Age 25 to 34 years,Age 35 to 44 years,Age 45 to 54 years,Age 55 to 59 years,Age 60 to 64 years,Age 65 to 74 years,Age 75 to 84 years,Age 85 years and over,Median age,White,Hispanic or Latino,Black or African American,Asian,American Indian & Alaska Native,Native Hawaiian & Other Pacific Islander,Other (One Race),Two or More Races,Population(16 years and over),Civilian labor force,Employed,Unemployed,Armed Forces,Not in labor force,Percent Unemployed,Workers(16 years and over),Drove alone,Carpooled,Public transportation(excluding taxicab),Walked,Other means,Worked at home,Mean travel time,Civilian employed population(16 years and over),Management / business / science / arts,Service,Sales / office,Natural resources / construction / maintenance,Production / transportation / material moving,Civilian employed population(16 years and over),Agriculture / forestry / fishing / hunting / mining,Construction,Manufacturing,Wholesale trade,Retail trade,Transportation / warehousing / utilities,Information,Finance / insurance / real estate / rental / leasing,Professional / scientific / management / administrative / waste management services,Educational services / health care / social assistance,Arts / entertainment / recreation / accommodation / food services,Public administration,Other services,Civilian employed population(16 years and over),Private wage & salary workers,Government workers,Self-employed(in own not incorporated business),Unpaid family workers,Total households,"Income Less than $10,000","Income $10,000 to $14,999","Income $15,000 to $24,999","Income $25,000 to $34,999","Income $35,000 to $49,999","Income $50,000 to $74,999","Income $75,000 to $99,999","Income $100,000 to $149,999","Income $150,000 to $199,999","Income $200,000 or more",Median household income,Mean household income,Percentage of people whose income in the past 12 months is below the poverty level
0,77479,"74,514","24,980",33.06,"2,253.90",1.55,"36,489","38,025",38.60,39.20,"24,076","20,738","3,338",3.09,3.38,"24,980","24,076","20,367","3,709",904,"4,455","6,050","7,022","5,880","3,065","6,758","11,918","14,078","5,630","4,014","3,464","1,546",634,39.00,"33,503","6,736","5,100","27,059",124,23,166,"1,803","54,403","36,562","34,928","1,634",7,"17,834",4.5%,"34,397","27,774","3,570",657,115,320,"1,961",30.0 min.,"34,928","22,157","1,818","8,696",913,"1,344","34,928","1,977","1,371","3,058","1,701","3,677","1,571",643,"3,500","5,783","8,006","1,611",712,"1,318","34,928","28,925","3,758","2,148",97,"22,797",507,253,762,947,"1,528","2,533","3,117","4,694","3,634","4,822","$121,145","$140,474",4.6%
1,02886,"29,269","13,682",14.93,"1,960.42",2.18,"13,896","15,373",41.70,47.10,"12,929","7,426","5,503",2.23,2.93,"13,682","12,929","8,534","4,395",753,"1,351","1,484","1,564","1,562","1,574","3,538","3,736","4,604","2,076","1,903","2,603","2,133","1,141",44.60,"26,611",992,382,758,76,5,50,395,"24,944","16,568","15,046","1,522",49,"8,327",9.2%,"14,604","12,582","1,136",167,128,186,405,22.0 min.,"15,046","5,416","2,602","4,461","1,232","1,335","15,046",14,904,"1,476",490,"2,147",632,237,"1,815","1,221","3,479","1,324",710,597,"15,046","12,312","2,051",683,0,"12,904",752,559,"1,184","1,198","1,616","2,596","1,925","1,975",624,475,"$59,220","$74,269",6.6%
2,28134,"9,453","4,287",8.56,"1,104.32",0.04,"4,399","5,054",33.70,37.90,"3,953","2,303","1,650",2.37,3.10,"4,287","3,953","2,312","1,641",334,691,611,623,494,546,"1,604","1,504","1,339",427,447,524,378,265,36.00,"5,409","1,516","1,757",513,29,16,41,172,"7,368","5,288","4,732",556,12,"2,068"

In [41]:
final_df.dtypes

Zipcode                                                                                object
Total population                                                                       object
Housing units                                                                          object
Land area(sq. miles)                                                                   object
Density(people per sq. mile)                                                           object
Water area(sq. miles)                                                                  object
Male                                                                                   object
Female                                                                                 object
Median_Age_Male                                                                        object
Median_Age_Female                                                                      object
Total households                                            

In [42]:
final_df.columns

Index(['Zipcode', 'Total population', 'Housing units', 'Land area(sq. miles)',
       'Density(people per sq. mile)', 'Water area(sq. miles)', 'Male',
       'Female', 'Median_Age_Male', 'Median_Age_Female', 'Total households',
       'Family households', 'Nonfamily households', 'Average household size',
       'Average family size', 'Total housing units', 'Occupied housing units',
       'Owner-occupied', 'Renter-occupied', 'Vacant housing units',
       'Age Under 5 years', 'Age 5 to 9 years', 'Age 10 to 14 years',
       'Age 15 to 19 years', 'Age 20 to 24 years', 'Age 25 to 34 years',
       'Age 35 to 44 years', 'Age 45 to 54 years', 'Age 55 to 59 years',
       'Age 60 to 64 years', 'Age 65 to 74 years', 'Age 75 to 84 years',
       'Age 85 years and over', 'Median age', 'White', 'Hispanic or Latino',
       'Black or African American', 'Asian', 'American Indian & Alaska Native',
       'Native Hawaiian & Other Pacific Islander', 'Other (One Race)',
       'Two or More Races', 'P

In [43]:
At_Home_stores_final = pd.merge(final_df, At_Home_stores, how = 'left', on = 'Zipcode')

In [44]:
At_Home_stores_final

,Zipcode,Total population,Housing units,Land area(sq. miles),Density(people per sq. mile),Water area(sq. miles),Male,Female,Median_Age_Male,Median_Age_Female,Total households,Family households,Nonfamily households,Average household size,Average family size,Total housing units,Occupied housing units,Owner-occupied,Renter-occupied,Vacant housing units,Age Under 5 years,Age 5 to 9 years,Age 10 to 14 years,Age 15 to 19 years,Age 20 to 24 years,Age 25 to 34 years,Age 35 to 44 years,Age 45 to 54 years,Age 55 to 59 years,Age 60 to 64 years,Age 65 to 74 years,Age 75 to 84 years,Age 85 years and over,Median age,White,Hispanic or Latino,Black or African American,Asian,American Indian & Alaska Native,Native Hawaiian & Other Pacific Islander,Other (One Race),Two or More Races,Population(16 years and over),Civilian labor force,Employed,Unemployed,Armed Forces,Not in labor force,Percent Unemployed,Workers(16 years and over),Drove alone,Carpooled,Public transportation(excluding taxicab),Walked,Other means,Worked at home,Mean travel time,Civilian employed population(16 years and over),Management / business / science / arts,Service,Sales / office,Natural resources / construction / maintenance,Production / transportation / material moving,Civilian employed population(16 years and over),Agriculture / forestry / fishing / hunting / mining,Construction,Manufacturing,Wholesale trade,Retail trade,Transportation / warehousing / utilities,Information,Finance / insurance / real estate / rental / leasing,Professional / scientific / management / administrative / waste management services,Educational services / health care / social assistance,Arts / entertainment / recreation / accommodation / food services,Public administration,Other services,Civilian employed population(16 years and over),Private wage & salary workers,Government workers,Self-employed(in own not incorporated business),Unpaid family workers,Total households,"Income Less than $10,000","Income $10,000 to $14,999","Income $15,000 to $24,999","Income $25,000 to $34,999","Income $35,000 to $49,999","Income $50,000 to $74,999","Income $75,000 to $99,999","Income $100,000 to $149,999","Income $150,000 to $199,999","Income $200,000 or more",Median household income,Mean household income,Percentage of people whose income in the past 12 months is below the poverty level,Name,Address,State,City,In_store_pickup,Curbside_pickup,local_delivery_pickup
0,77479,"74,514","24,980",33.06,"2,253.90",1.55,"36,489","38,025",38.60,39.20,"24,076","20,738","3,338",3.09,3.38,"24,980","24,076","20,367","3,709",904,"4,455","6,050","7,022","5,880","3,065","6,758","11,918","14,078","5,630","4,014","3,464","1,546",634,39.00,"33,503","6,736","5,100","27,059",124,23,166,"1,803","54,403","36,562","34,928","1,634",7,"17,834",4.5%,"34,397","27,774","3,570",657,115,320,"1,961",30.0 min.,"34,928","22,157","1,818","8,696",913,"1,344","34,928","1,977","1,371","3,058","1,701","3,677","1,571",643,"3,500","5,783","8,006","1,611",712,"1,318","34,928","28,925","3,758","2,148",97,"22,797",507,253,762,947,"1,528","2,533","3,117","4,694","3,634","4,822","$121,145","$140,474",4.6%,Texas-Sugar Land,16960 Southwest Fwy,TX,Sugar Land,1.0,1.0,1.0
1,02886,"29,269","13,682",14.93,"1,960.42",2.18,"13,896","15,373",41.70,47.10,"12,929","7,426","5,503",2.23,2.93,"13,682","12,929","8,534","4,395",753,"1,351","1,484","1,564","1,562","1,574","3,538","3,736","4,604","2,076","1,903","2,603","2,133","1,141",44.60,"26,611",992,382,758,76,5,50,395,"24,944","16,568","15,046","1,522",49,"8,327",9.2%,"14,604","12,582","1,136",167,128,186,405,22.0 min.,"15,046","5,416","2,602","4,461","1,232","1,335","15,046",14,904,"1,476",490,"2,147",632,237,"1,815","1,221","3,479","1,324",710,597,"15,046","12,312","2,051",683,0,"12,904",752,559,"1,184","1,198","1,616","2,596","1,925","1,975",624,475,"$59,220","$74,269",6.6%,Rhode Island-Warwick North,650 Bald Hill Rd,RI,Warwick,1.0,1.0,1.0
2,28134,"9,453","4,287",8.56,"1,104.32",0.04,"4,399","5,054",33.70,37.90,"3,953","2,

In [45]:
# # saving the data to csv
# At_Home_stores_final.to_csv('At_Home_Stores_final.csv',index = False)
# AHG_data = pd.read_csv('At_Home_Stores_final.csv')

In [46]:
AHG_sales_data.head()

,Year,DMA_DESC,STORE ADDRESS,CITY,STATE,ZIPCODE,SIMULATED_SALES
0,2017,Memphis,5280 Summer Ave,Memphis,TN,38122,1.158712e+07
1,2017,Atlanta,1605 Buford Highway,BUFORD,GA,30518,9.483031e+06
2,2017,Toledo,2244 S Reynolds Rd,TOLEDO,OH,43614,7.325231e+06
3,2017,Phoenix (Prescott),5000 S. Arizona Mills Circle,TEMPE,AZ,85282,6.567164e+06
4,2017,Memphis,7697 Winchester Road,Memphis,TN,38125,6.803906e+06


In [47]:
AHG_sales_data.Year.value_counts()

2020    211
2019    208
2018    176
2017    148
Name: Year, dtype: int64

In [48]:
AHG_sales_data.rename(columns = {'ZIPCODE':'Zipcode'}, inplace = True)

In [49]:
At_Home_stores_final['Zipcode'] = At_Home_stores_final['Zipcode'].apply(str)

In [50]:
AHG_sales_data.drop(columns= ['DMA_DESC','CITY','STATE','STORE ADDRESS'], inplace = True)

In [51]:
AHG_data_final = pd.merge(AHG_sales_data,At_Home_stores_final,how = 'left', on = 'Zipcode')

In [52]:
AHG_data_final.head()

,Year,Zipcode,SIMULATED_SALES,Total population,Housing units,Land area(sq. miles),Density(people per sq. mile),Water area(sq. miles),Male,Female,Median_Age_Male,Median_Age_Female,Total households,Family households,Nonfamily households,Average household size,Average family size,Total housing units,Occupied housing units,Owner-occupied,Renter-occupied,Vacant housing units,Age Under 5 years,Age 5 to 9 years,Age 10 to 14 years,Age 15 to 19 years,Age 20 to 24 years,Age 25 to 34 years,Age 35 to 44 years,Age 45 to 54 years,Age 55 to 59 years,Age 60 to 64 years,Age 65 to 74 years,Age 75 to 84 years,Age 85 years and over,Median age,White,Hispanic or Latino,Black or African American,Asian,American Indian & Alaska Native,Native Hawaiian & Other Pacific Islander,Other (One Race),Two or More Races,Population(16 years and over),Civilian labor force,Employed,Unemployed,Armed Forces,Not in labor force,Percent Unemployed,Workers(16 years and over),Drove alone,Carpooled,Public transportation(excluding taxicab),Walked,Other means,Worked at home,Mean travel time,Civilian employed population(16 years and over),Management / business / science / arts,Service,Sales / office,Natural resources / construction / maintenance,Production / transportation / material moving,Civilian employed population(16 years and over),Agriculture / forestry / fishing / hunting / mining,Construction,Manufacturing,Wholesale trade,Retail trade,Transportation / warehousing / utilities,Information,Finance / insurance / real estate / rental / leasing,Professional / scientific / management / administrative / waste management services,Educational services / health care / social assistance,Arts / entertainment / recreation / accommodation / food services,Public administration,Other services,Civilian employed population(16 years and over),Private wage & salary workers,Government workers,Self-employed(in own not incorporated business),Unpaid family workers,Total households,"Income Less than $10,000","Income $10,000 to $14,999","Income $15,000 to $24,999","Income $25,000 to $34,999","Income $35,000 to $49,999","Income $50,000 to $74,999","Income $75,000 to $99,999","Income $100,000 to $149,999","Income $150,000 to $199,999","Income $200,000 or more",Median household income,Mean household income,Percentage of people whose income in the past 12 months is below the poverty level,Name,Address,State,City,In_store_pickup,Curbside_pickup,local_delivery_pickup
0,2017,38122,1.158712e+07,"25,270","11,315",7.54,"3,351.46",0.00,"12,827","12,443",30.90,33.60,"9,791","5,787","4,004",2.58,3.25,"11,315","9,791","5,345","4,446","1,524","2,282","1,868","1,559","1,537","2,035","4,448","3,372","3,173","1,442","1,018","1,200",901,435,32.20,"12,732","5,590","5,754",663,43,6,53,429,"19,359","13,232","11,722","1,510",0,"6,127",11.4%,"11,422","8,522","2,074",190,111,254,271,22.0 min.,"11,722","2,771","2,384","3,078","1,893","1,596","11,722",67,"1,516",817,363,"1,675",893,166,529,"1,551","1,900","1,356",247,642,"11,722","9,791","1,023",901,7,"9,606",856,638,"1,734","1,475","1,697","1,674",878,592,29,33,"$35,449","$43,778",25.9%,Tennessee-Memphis,5280 Summer Ave,TN,Memphis,1.0,0.0,0.0
1,2017,30518,9.483031e+06,"45,211","16,972",39.74,"1,137.67",7.20,"22,227","22,984",33.50,35.60,"15,468","11,653","3,815",2.91,3.36,"16,972","15,468","11,838","3,630","1,504","3,850","4,015","3,640","3,017","2,160","6,157","7,923","6,467","2,303","1,767","2,173","1,229",510,34.70,"27,941","8,506","4,833","3,017",67,14,101,732,"33,053","23,313","21,390","1,923",14,"9,726",8.2%,"20,937","16,291","2,689",151,71,334,"1,401",30.1 min.,"21,390","7,741","3,389","6,228","1,934","2,098","21,390",77,"1,627","2,516","1,157","2,915",864,769,"1,710","2,642","3,146","2,631",300,"1,036","21,390","18,451","1,802","1,105",32,"14,651",618,598,"1,222","1,216","2,065","2,330","2,461","2,549",989,603,"$66,111","$82,765",14.9%,Georgia-Buford,1605 Buford Hwy,GA,Buford,1.0,1.0,1.0
2,2017,43614,7.325231e+06,"29,290","14,667",9.03,"3,243.63",0.36,"13,740",

In [53]:
AHG_data_final.shape

(743, 105)

In [54]:
AHG_data_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 743 entries, 0 to 742
Columns: 105 entries, Year to local_delivery_pickup
dtypes: float64(4), int64(1), object(100)
memory usage: 615.3+ KB


In [55]:
AHG_data_final.dtypes

Year                                                                                     int64
Zipcode                                                                                 object
SIMULATED_SALES                                                                        float64
Total population                                                                        object
Housing units                                                                           object
Land area(sq. miles)                                                                    object
Density(people per sq. mile)                                                            object
Water area(sq. miles)                                                                   object
Male                                                                                    object
Female                                                                                  object
Median_Age_Male                                   

In [56]:
# saving the combined final data to csv
AHG_data_final.to_csv('AHG_data_final.csv', index = False)